In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [2]:
(train_imgs, train_labs), (test_imgs, test_labs) = datasets.cifar10.load_data()

train_imgs, test_imgs = train_imgs / 255.0, test_imgs / 255.0

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [3]:
# Let's look at a one image
IMG_INDEX = 99  # change this to look at other images

plt.imshow(train_imgs[IMG_INDEX] ,cmap=plt.cm.binary)
plt.xlabel(class_names[train_labs[IMG_INDEX][0]])
plt.show()

In [4]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          36928     
Total params: 56,320
Trainable params: 56,320
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(10))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                6

In [7]:
model.compile(optimizer = 'adam', loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True), metrics = ['accuracy'])

history = model.fit(train_imgs, train_labs, epochs = 10, validation_data = (test_imgs, test_labs))


Epoch 1/10
1563/1563 [==============================] - 27s 6ms/step - loss: 1.5124 - accuracy: 0.4476 - val_loss: 1.2539 - val_accuracy: 0.5527
Epoch 2/10
1563/1563 [==============================] - 9s 5ms/step - loss: 1.1434 - accuracy: 0.5960 - val_loss: 1.0686 - val_accuracy: 0.6229
Epoch 3/10
1563/1563 [==============================] - 8s 5ms/step - loss: 0.9866 - accuracy: 0.6539 - val_loss: 1.0267 - val_accuracy: 0.6355
Epoch 4/10
1563/1563 [==============================] - 8s 5ms/step - loss: 0.8872 - accuracy: 0.6888 - val_loss: 0.9322 - val_accuracy: 0.6752
Epoch 5/10
1563/1563 [==============================] - 9s 6ms/step - loss: 0.8108 - accuracy: 0.7145 - val_loss: 0.8941 - val_accuracy: 0.6918
Epoch 6/10
1563/1563 [==============================] - 9s 5ms/step - loss: 0.7471 - accuracy: 0.7383 - val_loss: 0.8510 - val_accuracy: 0.7048
Epoch 7/10
1563/1563 [==============================] - 8s 5ms/step - loss: 0.6942 - accuracy: 0.7561 - val_loss: 0.8580 - val_accuracy

In [8]:
test_loss, test_acc = model.evaluate(test_imgs,  test_labs, verbose=2)
print(test_acc)

313/313 - 1s - loss: 0.8647 - accuracy: 0.7122
0.7121999859809875


In [9]:
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
rotation_range=40,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True,
fill_mode='nearest')

test_img = train_imgs[20]
img = image.img_to_array(test_img)  
img = img.reshape((1,) + img.shape)  

i = 0

for batch in datagen.flow(img, save_prefix='test', save_format='jpeg'):
    plt.figure(i)
    plot = plt.imshow(image.img_to_array(batch[0]))
    i += 1
    if i > 4:  # show 4 images
        break

plt.show()


In [10]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
keras = tf.keras

In [11]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

In [12]:
get_label_name = metadata.features['label'].int2str  # creates a function object that we can use to get labels

for image, label in raw_train.take(5):
  plt.figure()
  plt.imshow(image)
  plt.title(get_label_name(label))

In [13]:
IMG_SIZE = 160 
def format_example(image, label):
  """
  returns an image that is reshaped to IMG_SIZE
  """
  image = tf.cast(image, tf.float32)
  image = (image/127.5) - 1
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image, label

In [14]:
train = raw_train.map(format_example)
validation = raw_validation.map(format_example)
test = raw_test.map(format_example)

In [15]:
for image, label in train.take(2):
  plt.figure()
  plt.imshow(image)
  plt.title(get_label_name(label))

In [16]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [17]:
base_model.summary()

Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 80, 80, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 80, 80, 32)   0           bn_Conv1[0][0]                   
_______________________________________________________________________________

In [18]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000

train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = validation.batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)

In [19]:
for image, _ in train_batches.take(1):
   pass

feature_batch = base_model(image)
print(feature_batch.shape)

(32, 5, 5, 1280)


In [20]:
base_model.trainable = False

In [21]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

In [22]:
prediction_layer = keras.layers.Dense(1)

In [23]:
model = tf.keras.Sequential([base_model, global_average_layer, prediction_layer])

In [24]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

C:\Users\aaron\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [25]:
initial_epochs = 3
validation_steps=20

loss0,accuracy0 = model.evaluate(validation_batches, steps = validation_steps)

20/20 [==============================] - 2s 31ms/step - loss: 0.7905 - accuracy: 0.4625


In [26]:
history = model.fit(train_batches,
                    epochs=initial_epochs,
                    validation_data=validation_batches)

acc = history.history['accuracy']
print(acc)

Epoch 1/3
582/582 [==============================] - 37s 60ms/step - loss: 0.2261 - accuracy: 0.9009 - val_loss: 0.0926 - val_accuracy: 0.9656
Epoch 2/3
582/582 [==============================] - 39s 66ms/step - loss: 0.0747 - accuracy: 0.9737 - val_loss: 0.0665 - val_accuracy: 0.9746
Epoch 3/3
582/582 [==============================] - 36s 61ms/step - loss: 0.0591 - accuracy: 0.9786 - val_loss: 0.0577 - val_accuracy: 0.9785
[0.9008597731590271, 0.9736700654029846, 0.9785599112510681]


In [27]:
model.save("dogs_vs_cats.h5")  # we can save the model and reload it at anytime in the future
new_model = tf.keras.models.load_model('dogs_vs_cats.h5')

C:\Users\aaron\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
